# Импорты

In [18]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
import random
import datetime
from datetime import datetime, timedelta
import datetime as DT
import pandas as pd


# к удаленной бд

host_2 = '77.245.113.192'
user_2 = '*'
password_2 = '*'
db_name_2 = '*'
port_2 = 5432

connection_2 = psycopg2.connect(
host=host_2,
    user=user_2,
    password=password_2,
    database = db_name_2,
    port=port_2
)

connection_2.autocommit = True

#engine 

db_config = {'user': '*', 
'pwd': '*', 
'host': '77.245.113.192',
'port': 5432, 
'db': '*'} 


#Проверяем коннект

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db']) 

engine = create_engine(connection_string)

# Создание тестовой БД

Ради удобства создам таблички в базе данных с наполнением подобным тестовому заданию (как я его понимаю) для тестирования работоспособности запросов, у меня уже есть своя БД, в ней и создам временно таблички

In [13]:
# Табличка opers
with connection_2.cursor() as cursor:
    cursor.execute(

'''
    CREATE TABLE opers(
    viol_oper_id bigint PRIMARY KEY,
    tr_viol_id bigint,  
    oper_code integer,    
    refuse_code integer,    
    isp_id bigint,    
    date_oper timestamp(0),     
    viol_datetime timestamp(0)  
        
    )
    '''
    )

In [14]:
# Табличка viols

with connection_2.cursor() as cursor:
    cursor.execute(

'''
    CREATE TABLE viols(
    tr_viol_id bigint PRIMARY KEY,
    viol_code integer,  
    time_check timestamp(0),    
    stage_code integer,    
    refuse_code integer,    
    camera_id integer 
       
    )
    '''
    )

# Наполнение табличек

In [4]:
# Получим список дат с 01.01.2018 по 31.12.2018
start_date = DT.datetime(2018, 1, 1)
end_date = DT.datetime(2018, 12, 31)

res = pd.date_range(
    min(start_date, end_date),
    max(start_date, end_date)
).strftime('%Y-%m-%d').tolist()

In [8]:
# Заполним поле viol_code (xxx), tr_viol_id(xxx), stage_code(0,5), refuse_code(0,9), camera_id(xxx) случайными значениями
df_viols = {}
df_viols['tr_viol_id'] = []
df_viols['viol_code'] = []
df_viols['time_check'] = []
df_viols['stage_code'] = []
df_viols['refuse_code'] = []
df_viols['camera_id'] = []

df_viols['tr_viol_id'] = range(1,30001)
for _ in range(30000):
    viol_code = ''
    tr_viol_id = ''
    camera_id = ''
        
    for _ in range (3):
        viol_code += str(random.randint(1,9))
        tr_viol_id += str(random.randint(1,9))
        camera_id += str(random.randint(1,9))
    df_viols['viol_code'].append(int(viol_code))
    
    df_viols['camera_id'].append(int(camera_id))
    df_viols['refuse_code'].append(random.randint(0,9))
    df_viols['stage_code'].append(random.randint(0,5))
    #В даты возьмемм случайную дату из нашего списка и добавим  к ней случайное количество часов минут и секунд
    dt = datetime.strptime(random.choice(res), "%Y-%m-%d")+ timedelta(hours=random.randint(0,23))\
    + timedelta(minutes=random.randint(0,60)) + timedelta(seconds=random.randint(0,60))
    df_viols['time_check'].append(dt)

df_viols = pd.DataFrame(df_viols)
df_viols

,tr_viol_id,viol_code,time_check,stage_code,refuse_code,camera_id
0,1,739,2018-01-26 06:30:29,2,5,916
1,2,935,2018-03-20 01:56:09,2,0,453
2,3,216,2018-04-21 10:45:31,2,5,377
3,4,816,2018-08-01 12:21:33,3,3,352
4,5,337,2018-10-30 08:42:35,1,0,547
...,...,...,...,...,...,...
29995,29996,294,2018-05-18 08:38:59,0,5,465
29996,29997,414,2018-09-17 03:48:41,2,0,689
29997,29998,644,2018-08-25 01:07:39,4,6,431
29998,29999,658,2018-04-21 19:58:59,0,1,145


In [9]:
df_opers = {}
df_opers['viol_oper_id'] = []
df_opers['tr_viol_id'] = []
df_opers['oper_code'] = []
df_opers['refuse_code'] = []
df_opers['isp_id'] = []
df_opers['date_oper'] = []
df_opers['viol_datetime'] = []

# примем что поля в таблицах совпадают, однако премешаем для интереса
df_opers['tr_viol_id'] = df_viols['tr_viol_id'].copy()
random.shuffle(df_opers['tr_viol_id'])

#каждому нарушению соответсвует своя дата, сделаем что-бы тоже совподало
for i in df_opers['tr_viol_id'].values:
    df_opers['viol_datetime'].append(df_viols['time_check'][i-1])


#Предположим,что время операции обработки нарушения - в течение часа, прибавим случайное количество минут и секунд к дате нарушения 
for date in df_opers['viol_datetime']:
    df_opers['date_oper'].append(date+timedelta(minutes=random.randint(0,59)) + timedelta(seconds=random.randint(0,59)) )
    
#viol_oper_id, oper_code(0,31), refuse_code(0,9), isp_id(xxx) для этих полей зададим случайные значения
df_opers['viol_oper_id'] = range(1, 30001)
for _ in range(30000):    
    
    isp_id = ''      
    for _ in range (3):
        isp_id += str(random.randint(1,9))
        
    df_opers['oper_code'].append(random.randint(0,31))
    df_opers['refuse_code'].append(random.randint(0,9))
    df_opers['isp_id'].append(int(isp_id))
    

df_opers = pd.DataFrame(df_opers)
df_opers.head(5)

,viol_oper_id,tr_viol_id,oper_code,refuse_code,isp_id,date_oper,viol_datetime
0,1,29753,9,0,645,2018-05-21 15:47:56,2018-05-21 15:00:00
1,2,3808,9,6,987,2018-02-12 23:28:07,2018-02-12 22:28:17
2,3,25989,13,9,441,2018-08-26 03:54:49,2018-08-26 03:19:26
3,4,5523,10,7,898,2018-03-29 15:22:56,2018-03-29 14:23:49
4,5,10766,0,8,942,2018-06-17 11:47:15,2018-06-17 10:55:36


Теперь у нас есть 30000 строк для экспериментов, но для начала запишем все в нашу тестовую БД

In [15]:
SQL_insert = SQL_insert = "INSERT INTO opers (viol_oper_id, tr_viol_id, oper_code, refuse_code, isp_id, date_oper,\
viol_datetime) VALUES (%s, %s, %s, %s, %s, %s, %s);"
for q in range (len(df_opers)):
    values_ = []
    for value in df_opers.loc[q].values:
        values_.append(str(value))
    with connection_2.cursor() as cursor:
            cursor.execute(
            SQL_insert, [*values_]
            )

In [16]:
SQL_insert = SQL_insert = "INSERT INTO viols (tr_viol_id, viol_code, time_check, stage_code, refuse_code, camera_id)\
VALUES (%s, %s, %s, %s, %s, %s);"
for q in range (len(df_viols)):
    values_ = []
    for value in df_viols.loc[q].values:
        values_.append(str(value))
    with connection_2.cursor() as cursor:
            cursor.execute(
            SQL_insert, [*values_]
            )

Готово, можно проступать к запросам

# Запрос

Подготовить выгрузку, содержащую информацию, за период нарушения ПДД с 2018-04-01 по 2018-04-20 с детализацией до даты нарушения. Выгрузка должна содержать атрибуты:
   - Дата нарушения ПДД;
   - Число нарушений ПДД всего;
   - Число нарушений ПДД у которых результирующий код обработки нарушения ПДД равен 0;
   - Число нарушений ПДД по которым была операция обработки тип 29;
   - Число нарушений ПДД по которым не было операции обработки тип 29, а результирующий код обработки нарушения ПДД равен 0;


In [47]:
query = '''

WITH n_1 AS 
(SELECT CAST(viol_datetime AS DATE) AS date,
       COUNT(viol_datetime) AS VIOLS_COUNT
FROM opers
WHERE viol_datetime >= '2018-04-01'
  AND viol_datetime < '2018-04-21'
GROUP BY date
ORDER BY date ASC),

n_2 AS 
(SELECT CAST(time_check AS DATE) AS date,
       COUNT(refuse_code) AS zero_refuse_code_count       
FROM viols
WHERE time_check >= '2018-04-01'
  AND time_check <= '2018-04-20'
  AND refuse_code = 0
GROUP BY date
ORDER BY date ASC),

n_3 AS 
(SELECT CAST(viol_datetime AS DATE) AS date,
       COUNT(oper_code) AS oper_code_29       
FROM opers
WHERE viol_datetime >= '2018-04-01'
  AND viol_datetime <= '2018-04-20'
  AND oper_code = 29
GROUP BY date
ORDER BY date ASC),

n_4 AS 
(SELECT CAST(viols.time_check AS DATE) AS date, COUNT(*) AS oper_code_not_29_and_zero_refuse_code
FROM viols
JOIN opers ON viols.tr_viol_id = opers.tr_viol_id
WHERE viols.time_check >= '2018-04-01'
  AND viols.time_check <= '2018-04-20'
  AND opers.oper_code != 29
  AND viols.refuse_code = 0
  GROUP BY date
  ORDER BY date ASC)
  
SELECT n_1.date AS date, n_1.viols_count, n_2.zero_refuse_code_count,
       n_3.oper_code_29, n_4.oper_code_not_29_and_zero_refuse_code
FROM n_1
LEFT JOIN n_2 ON n_1.date = n_2.date
LEFT JOIN n_3 ON n_1.date = n_3.date
LEFT JOIN n_4 ON n_1.date = n_4.date

'''
pd.io.sql.read_sql(query, con = engine)

,date,viols_count,zero_refuse_code_count,oper_code_29,oper_code_not_29_and_zero_refuse_code
0,2018-04-01,86,12.0,3.0,12.0
1,2018-04-02,76,6.0,NaN,6.0
2,2018-04-03,78,7.0,3.0,6.0
3,2018-04-04,70,4.0,2.0,4.0
4,2018-04-05,79,10.0,NaN,10.0
5,2018-04-06,96,9.0,4.0,9.0
6,2018-04-07,94,5.0,4.0,4.0
7,2018-04-08,92,9.0,3.0,8.0
8,2018-04-09,75,3.0,1.0,3.0
9,2018-04-10,74,6.0,3.0,6.0


## Пояснение

В рамках данной задачи я воспользовался временными таблицами, так как считаю что такой вид запросов более читаемый чем подзапросы и не уступает в скорости, так же такие запросы гораздо проще поддаются отладке и есть возможность использовать временные таблицы как отдельные элементы для анализа, либо в разных сочетаниях друг с другом.

Каждая отдельная временная таблица решает один пункт тз, затем просто соединяем их в сводную таблицу используя Left join, что-бы не потерялись данные, когда в заданный день нет объектов соответствующих условиям